In [37]:
from os import getenv
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

In [38]:
load_dotenv()

True

In [5]:
neo4j_url= getenv("NEO4J_URL")
neo4j_user = getenv("NEO4J_USER")
neo4j_password = getenv("NEO4J_PW")
neo4j_database = getenv("NEO4J_DB")

In [6]:
graph = Neo4jGraph(neo4j_url, neo4j_user, neo4j_password, neo4j_database)

In [7]:
print(graph.schema)

Node properties:
Movie {certificate: STRING, rating: FLOAT, run_time: INTEGER, tagline: STRING, title: STRING, year: INTEGER}
Person {name: STRING}
Actor {name: STRING}
Director {name: STRING}
Writer {name: STRING}
Genre {name: STRING}
Province {year: INTEGER, name: STRING, average_population: FLOAT, population_density: INTEGER, population_growth_rate: FLOAT}
region {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:BELONGS_TO]->(:Genre)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:WROTE]->(:Movie)
(:Actor)-[:ACTED_IN]->(:Movie)
(:Actor)-[:WROTE]->(:Movie)
(:Actor)-[:DIRECTED]->(:Movie)
(:Director)-[:DIRECTED]->(:Movie)
(:Director)-[:WROTE]->(:Movie)
(:Director)-[:ACTED_IN]->(:Movie)
(:Writer)-[:DIRECTED]->(:Movie)
(:Writer)-[:WROTE]->(:Movie)
(:Writer)-[:ACTED_IN]->(:Movie)


In [39]:
openAI_api = getenv("OPEN_AI_2")

In [40]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key = openAI_api, temperature = 0)

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True,allow_dangerous_requests=True)


In [42]:
chain.invoke({"query": "all information about Ha Noi"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Province {name: 'Ha Noi'}) RETURN p

Full Context:
[]

> Finished chain.


{'query': 'all information about Ha Noi', 'result': "I don't know the answer."}

In [43]:
questions = ["Tất cả thông tin về hà nội"]

for question in questions:
    print("\n\n" + question)
    print(chain.invoke(question)["result"])



Tất cả thông tin về hà nội


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Province {name: 'Hà Nội'}) RETURN p

Full Context:
[{'p': {'average_population': 8093.9, 'year': 2019, 'population_density': 2410, 'name': 'Hà Nội', 'population_growth_rate': 2.27}}, {'p': {'average_population': 8246.6, 'year': 2020, 'population_density': 2455, 'name': 'Hà Nội', 'population_growth_rate': 1.89}}]

> Finished chain.
Hà Nội có dân số trung bình là 8,093.9 vào năm 2019 với mật độ dân số là 2,410 và tỷ lệ tăng trưởng dân số là 2.27%. Vào năm 2020, dân số trung bình của Hà Nội là 8,246.6, mật độ dân số là 2,455 và tỷ lệ tăng trưởng dân số giảm xuống còn 1.89%.
